# Heuristic_Procedure



The reassign procedure is not implemented. This part of the code is not completed and can be omitted. In case of using it, it must be reviewed first. There are aspects that need to be check, for example the value of vnf_type (equal to f_type or next_vnf type??)


In [1]:
import pickle
import random
import time
import copy
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
def import_data(dir_data, file_name):
    """THis function imports the data that it is in dir_data+file_name"""
    with open (dir_data+file_name, "rb") as fp:
        return pickle.load(fp)

In [3]:
def export_data(file_name, file_data):
    """Esta funcion me permite salvar en un fichero con nombbre file_data la información contenida en file_data"""
    with open(file_name, "wb") as fp:
        pickle.dump(file_data, fp)
    return

## Importing data

In [114]:
# Path to initial solution
dir_data = 'path_initial_solution'  


# Number of users (i.e., PDU sessions or SFCR)
S = import_data(dir_data, "S") 

# Session information
N_sfc_type = import_data(dir_data, "N_sfc_type_S"+str(S)) 
Dict_Tsfc_s = import_data(dir_data, "Dict_Tsfc_s_S"+str(S)) 
Dict_C_s = import_data(dir_data, "Dict_C_s_S"+str(S)) 
Dict_Lreq_s = import_data(dir_data, "Dict_Lreq_s_S"+str(S)) 
Dict_beta_s = import_data(dir_data, "Dict_beta_s_S"+str(S)) 
Source_BS = import_data(dir_data, "Source_BS_S"+str(S)) 


# Nodes
# Candidate locations
Nc = import_data(dir_data, "ListMECInstalled")        # candidate coordenadas
Nc = sorted(Nc , key=lambda k: [k[1], k[0]])
capacity_nc = import_data(dir_data, "Dict_C_c") 
Available_nc = list(capacity_nc.keys())               # inicialmente todos los candidatos tienen capacidad
# access nodes
Nr = import_data(dir_data, "BS_pos")
Nr = sorted(Nr , key=lambda k: [k[1], k[0]])
# Set of all nodes. 
N = Nc + Nr 


# paths data 
Edges = import_data(dir_data,"Edges")                               # dict with key nodes_id (en & ran) and value latency
Dict_Links_Capacity = import_data(dir_data,"Dict_Links_Capacity")   # dict with key nodes_id (en & ran) and value bandwidth
Paths = import_data(dir_data,"Paths")                     
Paths_links_mapping = import_data(dir_data,'Paths_links_mapping')   # H^p_uv: 1 if path p ∈ P is mapped to physical link (u,v) ∈ E


# VNFs Data
Dict_C_t = import_data(dir_data,'Dict_C_t')             # Processing capacity of VNF of type t ∈ T         
Dict_D_proc = import_data(dir_data,'Dict_D_proc')       # d_t: Processing delay of VNF of type t ∈ T
M_t_imported = import_data(dir_data,'M_t')              # M_t: Maximum number of instances of type t ∈ T. Compute upon service demands and VNF capacity
T = import_data(dir_data,'T')                           # T: Types of VNFs
T_extended = T+1                                        # Extending T dataset to include RANs as an special VNF type
T_dn = import_data(dir_data,'T_dn')                     # Processing time of data network nodes


### Preparing the Data

In [ ]:
# weightFactors List
WeightFactors = [0.4, 0.4, 0.2]

# Normalization parameters
max_latency = 100  #PDT DEFINIR
max_upf = round(sum(capacity_nc.values())/min(capacity_ct.values()))
max_server = len(capacity_nc)
NormalizationValues = [max_server, max_upf, max_latency]

In [115]:
Cap_aupf = 0
Cap_iupf = 0
Cap_miupf = 0
t1, t2, t3 = 0,0,0

for user in range(S):
    Cap_aupf +=  Dict_C_s[user+1]
    if Dict_Tsfc_s[user+1]==3:
        Cap_aupf +=  Dict_C_s[user+1]
        Cap_miupf +=  Dict_C_s[user+1]
        t3 +=1
    elif Dict_Tsfc_s[user+1]==2:  
        Cap_iupf +=  Dict_C_s[user+1]
        t2 +=1
    else:  
        t1 +=1

In [118]:
# Splitting the set of links between candidate nodes and ran-candidates

Edge_candidates = {}
Edge_rans_candidates = {}

for k,v in Edges.items():
    if k[0] <= len(Nc) and k[1] <= len(Nc):
        Edge_candidates[k] = v
    else:
        Edge_rans_candidates[k] = v

if len(Edge_candidates) + len(Edge_rans_candidates) != len(Edges):
    print('Error: There was something wrong in the splitting process')
    raise

##### Defining the general parameters for each type SFC

In [119]:
#  Parameters for each type of SFCR

if N_sfc_type==3:                              # T: Number of VNFs types 1:PSA,  2:IUPF,3: M-IUPF, 4:RAN
    Ran_type = 4
    B_sfc_type = [1,1,2]                       # No. of branches in each type of SFC
    F_sfc_type = [[1,2],[1,2,3],[1, 2, 3, 4]]  # VNFs in each type of SFC
    T_sfc_type = [[1,4],[2,1,4],[3,1,1,4]]     # Type for each VNF in SFC 
    K_sfc_type = [[1,1],[1,1,1],[1,0,0,1]]     # Indicates if a VNF instance is common to all the branches or not
    P_fbs_sfc_type = [[[1,1]], 
                      [[1,1,1]], 
                      [[1,1,0,1],[1,0,1,1]]]   # Indicates if a VNF is present or not in a branch: Index[type,branch,vnf]
    O_sbfg_sfc_type  = [[[[0,0], [1,0]]],
                        [[[0,1,0],[0,0,0],[1,0,0]]],
                        [[[0,1,0,0],[0,0,0,0],[0,0,0,0],[1,0,0,0]],
                        [[0,0,1,0],[0,0,0,0],[0,0,0,0],[1,0,0,0]]]]  #Indicates if a VNF f goes before g in a branch

if N_sfc_type==2:                               # T: Number of VNFs types 1:PSA,  2:IUPF, 3:RAN
    Ran_type = 3
    B_sfc_type = [1,1]                          # No. of branches in each type of SFC
    F_sfc_type = [[1,2],[1,2,3]]                # VNFs in each type of SFC
    T_sfc_type = [[1,3],[2,1,3]]                # Type for each VNF in SFC 
    K_sfc_type = [[1,1],[1,1,1]]                # Indicates if a VNF instance is common to all the branches or not
    P_fbs_sfc_type = [[[1,1]], 
                      [[1,1,1]]]                # Indicates if a VNF is present or not in a branch: Index[type,branch,vnf]
    O_sbfg_sfc_type  = [[[[0,0], [1,0]]],
                        [[[0,1,0],[0,0,0],[1,0,0]]]]  #Indicates if a VNF f goes before g in a branch

### Defining PDU sessions  parameters

In [121]:
#Generating the data of each session PDU (demand, A-UPF number, BS source)
List_F_s = []
List_T_s = []
List_K_s = []
List_P_s = []
List_O_s = []
List_B_s = []
List_NumVNF_s = []
List_R_s = []  # It has for each session the tuple (s,f,t,m,c)
List_Lreq_s = []
List_C_s = []
List_beta_s = []
List_L_prop_s = []

for k,v in Dict_Tsfc_s.items():
    List_F_s.append(F_sfc_type[v-1])
    List_T_s.append(T_sfc_type[v-1])
    List_K_s.append(K_sfc_type[v-1])
    List_P_s.append(P_fbs_sfc_type[v-1])
    List_B_s.append(B_sfc_type[v-1])
    List_NumVNF_s.append(len(F_sfc_type[v-1]))
    List_R_s.append(Nr.index(Source_BS[k])+1+len(Nc))
    List_Lreq_s.append(Dict_Lreq_s[k])
    aux_list = []
    for b in P_fbs_sfc_type[v-1]:
        L_proc = Dict_D_proc[Ran_type]+T_dn
        for ind in range(len(b)):
            if b[ind]==1:
                L_proc += 2*Dict_D_proc[T_sfc_type[v-1][ind]]
        aux_list.append((Dict_Lreq_s[k]-L_proc)/2) 
    List_L_prop_s.append(aux_list)
    List_C_s.append(Dict_C_s[k])
    List_beta_s.append(Dict_beta_s[k])

In [ ]:
SessionInfo = pd.DataFrame()
SessionInfo['SessionId'] = list(Dict_Tsfc_s.keys())
SessionInfo['Latency'] = List_Lreq_s
SessionInfo['L_prop'] = List_L_prop_s
SessionInfo['Demand'] = List_C_s
SessionInfo['Bw'] = List_beta_s
SessionInfo['SourceBS'] = List_R_s 
SessionInfo['Branches'] = List_B_s
SessionInfo['VNFs'] = List_F_s 
SessionInfo['NumVNFs'] = List_NumVNF_s 
SessionInfo['Presence'] = List_P_s 
SessionInfo['Type'] = List_T_s 
SessionInfo['Common'] = List_K_s 

SessionInfo

,SessionId,Latency,L_prop,Demand,Bw,SourceBS,Branches,VNFs,NumVNFs,Presence,Type,Common
0,1,1000,[150.0],0.1,10,54,1,[1],1,[[1]],[1],[1]
1,2,900,"[50.0, 50.0]",0.2,20,45,2,"[1, 2, 3]",3,"[[1, 1, 0], [1, 0, 1]]","[3, 1, 1]","[1, 0, 0]"
2,3,900,[50.0],0.2,10,92,1,"[1, 2]",2,"[[1, 1]]","[2, 1]","[1, 1]"
3,4,900,"[50.0, 50.0]",0.2,20,79,2,"[1, 2, 3]",3,"[[1, 1, 0], [1, 0, 1]]","[3, 1, 1]","[1, 0, 0]"
4,5,900,[50.0],0.1,20,108,1,"[1, 2]",2,"[[1, 1]]","[2, 1]","[1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,900,[100.0],0.2,10,47,1,[1],1,[[1]],[1],[1]
196,197,900,[50.0],0.2,20,44,1,"[1, 2]",2,"[[1, 1]]","[2, 1]","[1, 1]"
197,198,1000,"[100.0, 100.0]",0.1,10,93,2,"[1, 2, 3]",3,"[[1, 1, 0], [1, 0, 1]]","[3, 1, 1]","[1, 0, 0]"
198,199,1000,[100.0],0.2,10,80,1,"[1, 2]",2,"[[1, 1]]","[2, 1]","[1, 1]"


### Auxiliar Functions

In [123]:
def critic_classifier(session_info, available_c):
    """ 
    This function classified the sessions as critic or not depending on their number of available candidates
    Input:
    - session_info: DF with info about sessions
    - available_c: List of available candidates for the VNf placement
    """
    
    List_critic_s = []     # 1 if critic, 0 otherwise
    List_Numcand_s = []
    List_candidates_s = []
    
    for index, row in session_info.iterrows():
        aux_list = determine_candidates(row['SourceBS'], row['L_prop'], available_c)

        List_Numcand_s.append(len(aux_list))
        List_candidates_s.append(aux_list)
        if len(row['Type'])==1:

            if len(aux_list)<=1:
                List_critic_s.append(1)
            else:
                List_critic_s.append(0)
                
        else:
            if len(aux_list)<=2:
                List_critic_s.append(1)
            else:
                List_critic_s.append(0)

    session_info['Critic'] = List_critic_s 
    session_info['NumCand'] = List_Numcand_s 
    session_info['Candidates'] = List_candidates_s 
    
    return session_info

In [ ]:
def determine_candidates(source_vnf, Lprop_vnf):
    """
    This fucntion determines the candidates for the placement of a given vnf given the propagation delay budget.
    Inputs:
    -source_vnf: The id of the source node (the previous vnf in the branch of the current vnf)
    -Lprop_vnf. the latency budget for the propagation delay    
    Output:
    - List_c: A list with the position of the candidates
    """      
    List_c = []
    
    for kp, vp in Paths.items():   # kp, vp: path, propagation delay
        if kp[0] == source_vnf and vp <= min(Lprop_vnf):
            if kp[1] not in List_c:
                List_c.append(kp[1]) 
                
    return List_c

In [125]:
def reassign_part1():
    """
    This part reassigns an existent VNf of an already assigned SFCR
    """

    Copy_ServersCap = copy.deepcopy(ServersCap)
    Copy_LinksCap = copy.deepcopy(LinksCap)
    Copy_OpenServers = copy.deepcopy(OpenServers)
    Copy_VNFInstances = copy.deepcopy(VNFInstances)             # the key is the type of VNF and the value is a list of instances id
    Copy_VNFCapacity = copy.deepcopy(VNFCapacity)               # dict of dict, main dict key=type, valueis a dict key=instance_id & value=available cap
    Copy_NodeVNFInstances = copy.deepcopy(NodeVNFInstances)     # dict of dict, main dict key=node id, valueis a dict key=vnf_type & value=instance id
    Copy_SFCMapping = copy.deepcopy(SFCMapping)                 # dict of dict, main dict key=pdu id, value= dict key=vnf_id(f)& value=type,instance_id,nodeId

    for k1,v1 in Dict_Unfeasiblecandidate.items():

        # Checking if failure is due to C17 constraint only
        if len(v1)==1 and ('C17'in v1 or 'C16' in v1):
            vnf_type = 1 #psa   

            # verify if there is a PSA in k1 with available capacity
            if len(NodeVNFInstances.get(k1, {}))!=0: 

                if len(NodeVNFInstances[k1].get(vnf_type, []))==0:  #no psa in k1, we should try to create one
                    if Copy_ServersCap[k1]>= capacity_ct[vnf_type]:
                        flag_create_psa = True
                    else:
                        flag_create_psa = False
                        print("NO PSA INSTANCE OR CAPACITY TO CREATE ONE, NO CREO Q ESTO PASE")
                else:
                    flag_create_psa = True
                    #verify if there is a psa with available capacity
                    for inst in NodeVNFInstances[k1][vnf_type]:
                        if VNFCapacity[vnf_type][inst] >= row['Demand']:
                            new_instance = inst
                            flag_create_psa = False
                            break
                    if flag_create_psa and (Copy_ServersCap[k1]< capacity_ct[vnf_type]): 
                        flag_create_psa = False
                        print("NO PSA INSTANCE con CAPACITY or capacity TO CREATE ONE, NO CREO Q ESTO PASE")
                        
            else:  
                # unopen server (no active server)
                Copy_OpenServers.append(k1)                     #update set of open servers
                flag_create_psa = True
          
            if flag_create_psa:
                Copy_ServersCap[k1]-= capacity_ct[vnf_type]     #update server capacity
                new_instance = len(Copy_VNFInstances.get(vnf_type, []))+1
                Copy_VNFInstances.setdefault(vnf_type,[]).append(new_instance)
                Copy_VNFCapacity[vnf_type] = VNFCapacity.setdefault(vnf_type,{})
                Copy_VNFCapacity[vnf_type][new_instance] = capacity_ct[vnf_type]
                Copy_NodeVNFInstances.setdefault(k1, {}).setdefault(vnf_type,[]).append(new_instance)
                
            # Searching for SFCR to reassign to this PSA instance
            candidate_near_k1 = {}
            for c in List_c:
                if c!=k1:
                    if (k1,c) in Edges.keys():
                        candidate_near_k1[c]=Edges[(k1,c)] 
            sort_candidate_near_k1 = sorted(candidate_near_k1.items(), key=lambda x: x[1])

            for nc in sort_candidate_near_k1:

                #Verifyng if the candidate has at least one PSA
                list_psa_nc = NodeVNFInstances.get(nc[0],{}).get(vnf_type, [])

                for psa in list_psa_nc:
                    for sfcr in VNFSFCAssigned[vnf_type][psa]:
                        for index, row_sfcr in SortedSessionInfo.loc[SortedSessionInfo['SessionId'] == sfcr].iterrows():
                            break
                        if row_sfcr['Demand']>=row['Demand'] and VNFCapacity[vnf_type][new_instance]>=row_sfcr['Demand']:
                            latency_sfcr = 0
                            possible_paths = []
                            for path in SFCMapping[sfcr]['Paths']:
                                latency_sfcr += Paths[path]
                                if path[1]==nc[0]:
                                    possible_paths.append(path)

                            if len(possible_paths)==1: # it's easy to find the source
                                source_sfcr = possible_paths[0][0]
                                #verifying  latency
                                latency_budget = row_sfcr['L_prop'][0] - latency_sfcr+Paths[possible_paths[0]]
                                flag_latency = False

                                for k,v in Paths.items():
                                    if k[0]==source_sfcr and k[1]==k1:
                                        if v <= latency_budget:
                                            flag_latency= True
                                            break

                                if flag_latency: # old instance in SFCMapping
                                    for vnf_id, mapp in SFCMapping[sfcr]['VNFs'].items():
                                        if mapp[2]==nc[0] and mapp[0]==vnf_type:
                                            old_instance =  mapp[1]

                                    # We should release links capacity before verfying the constraints
                                    for link,v in Paths_links_mapping[possible_paths[0]].items():
                                        if v==1: 
                                            #update link capacity
                                            Copy_LinksCap[link] += row_sfcr['Bw']

                                    List_const, Selected_path, VariableValues = verify_candidate(k1, source_sfcr, vnf_type, row_sfcr)

                                    if len(List_const)== 0 and Selected_path != None:   #Feasible candidate
                                        #updating sfcr data
                                        Copy_VNFCapacity[vnf_type][new_instance]-= row_sfcr['Demand']  # new location
                                        Copy_VNFCapacity[vnf_type][old_instance]+= row_sfcr['Demand']  # old location

                                        for link,v in Paths_links_mapping[Selected_path].items():
                                            if v==1:
                                                 #update link capacity
                                                Copy_LinksCap[link]-= row_sfcr['Bw']

                                        Copy_SFCMapping[row_sfcr['SessionId']]['VNFs'][vnf_id] = (vnf_type, 
                                                                                                  new_instance, 
                                                                                                  k1)
                                        Copy_SFCMapping[row_sfcr['SessionId']]['Paths'] = [Selected_path if x== possible_paths[0]
                                                                                           else x for x in Copy_SFCMapping[row_sfcr['SessionId']]
                                                                                           ['Paths']]

                                        return (Copy_ServersCap,Copy_LinksCap,Copy_OpenServers,Copy_VNFInstances,
                                                Copy_VNFCapacity,Copy_NodeVNFInstances, Copy_SFCMapping)

    print("This is an unfeasible case pending of analisis")
    
    return ServersCap, LinksCap, OpenServers, VNFInstances, VNFCapacity, NodeVNFInstances, SFCMapping

In [126]:
def verify_candidate(solution_, c, source_vnf, vnf_type, row):
    """
    this function verifies that none constarint is violated if vnf is placed at this location. 
    It also verifies the existence of a path with available capacity. In case of more than one path it returns 
    the shortest one
    Inputs:
    -c: candiate node(id)
    Outputs:
    -List_const: list with number of constraints violated. If no constraint is exceeded, then is empty
    -Selectd_path: selected path for the routing. default value=none
    -VariableValues: Binary Tuple (openserver, newinstance, delay). Default values [0,0,0]
    """

    List_const = []
    Selected_path = None
    VariableValues =  [0,0,0]
    
    # verifying the existence of a vnf instance of same type
    Instances_c = solution_['NodeVNFInstances'].get(c, {}).get(vnf_type, None)
    VNFCapFailure = False 
    
    if Instances_c != None:  #There is at least a VNf instance of the same type
        # Verify instances capacity
        VNFCapFailure = True
        for inst in Instances_c:
            if solution_['VNFCapacity'][vnf_type][inst] >= row['Demand']:
                VNFCapFailure = False
                break
                
    if Instances_c==None or VNFCapFailure: #There is not instances or they do not have capacity
        if solution_['ServersCap'][c]< capacity_ct[vnf_type]:   #the candidate does not have more capacity
            List_const.append('C1')
            if VNFCapFailure:
                List_const.append('C2')
        else:
            # I should create a new instance of vnf_type, but before we need to verify the max num instances
            if len(solution_['VNFInstances'].get(vnf_type, []))+1>M_t[vnf_type]:
                List_const.append('C4')
    
    # verifying UPF constraints
    if vnf_type==1:  # It's a PSA UPF
        for v in solution_['SFCMapping'].get(row['SessionId'], {}).get('VNFs', {}).values():
            # Anti-affinity constraint:
            if v[0]==vnf_type and v[2]==c:   # v--> (type, instance, c)
                List_const.append('C16')
                break
            # PSA_IUPF constraint:
            elif v[0]==2 and v[2]==c:   # v--> (type, instance, c)
                List_const.append('C17')
                break
                
    # finding the shortest path with available capacity
    pathid=[]
    pathdelay=[]
    for k,v in Paths.items():
        if k[0]==source_vnf and k[1]==c:
            pathid.append(k)
            pathdelay.append(v)
    SortedPaths = [x for _,x in sorted(zip(pathdelay, pathid))]     # sorting the paths by their prop delay        
    
    #checking path capacity
    for path in SortedPaths:
        LinkCapFailure = False
        for link,v in Paths_links_mapping[path].items():
            if v==1: #verify link capacity
                if solution_['LinksCap'][link]<row['Bw']:
                    LinkCapFailure = True
                    break
        if not LinkCapFailure:
            Selected_path = path
            break
            
    if Selected_path==None: 
        List_const.append('C3')
    
    if len(List_const)==0:
        # Determing the VariableValues
        if solution_['NodeVNFInstances'].get(c, None)== None:
            VariableValues[0] = 1
            VariableValues[1] = 1
        elif VNFCapFailure or Instances_c == None:
            VariableValues[1] = 1
        VariableValues[2] = Paths[Selected_path]/100
        
    return List_const, Selected_path, VariableValues

In [127]:
def simulating_vnfdeployment(solution_2, c, vnf_id, vnf_type, Selected_path, VariableValues, row):
    """
    This function will simulate the deployment of vnf in c to analize the deployment posibilities for the next 
    vnf if the current location is selected.
    1-To simulate the deployment we will create a copy of the sets and add the mapping of vnf to links and nodes
    2-Find next VNF in the branch 
    3-Take c as source for vnf+1 and determine the candidates
    3-Evaluate each candidate
    4-Select the best candidate
    """  
    
    #Updating the Sets
    if VariableValues[0]==1:                    
        # Opening of a new server   
        solution_2['OpenServers'].append(c)
        
    if VariableValues[1]==1:
        # create a new instance of vnf_type   
        solution_2['ServersCap'][c] -= capacity_ct[vnf_type]
        instance = len(solution_2['VNFInstances'].get(vnf_type, []))+1
        solution_2['VNFInstances'].setdefault(vnf_type,[]).append(instance)
        solution_2['VNFCapacity'].setdefault(vnf_type,{})[instance] = capacity_ct[vnf_type]
        solution_2['NodeVNFInstances'].setdefault(c, {}).setdefault(vnf_type,[]).append(instance)
    
    else:  
        # find instance with capacity
        for instance in solution_2['NodeVNFInstances'][c][vnf_type]:
            if solution_2['VNFCapacity'][vnf_type][instance] >= row['Demand']:
                break

    # update capacity
    solution_2['VNFCapacity'][vnf_type][instance] = round(solution_2['VNFCapacity'][vnf_type][instance]-row['Demand'],2)
    solution_2['SFCMapping'].setdefault(row['SessionId'], {}).setdefault('VNFs', {})[vnf_id] = (vnf_type, instance, c)
    solution_2['SFCMapping'].setdefault(row['SessionId'], {}).setdefault('Paths', []).append(Selected_path)
    solution_2['VNFSFCAssigned'].setdefault(vnf_type,{}).setdefault(instance,[]).append(row['SessionId'])

    for link,v in Paths_links_mapping[Selected_path].items():
        if v==1: 
            #update link capacity
            solution_2['LinksCap'][link]-= row['Bw']
    
    return solution_2

In [136]:
def evaluate_candidates(solution_, candidates_set, source, v_type, row, candidates_sfcr, sfcr_u ):
    """
    Determines which candidates in the set are feasible and which one not for the deployment of the selected vnf
    sfcr_u: Number of SFCR remaining for assignment
    """

    unfeasible_candidates = {} 
    feasible_cadidates = {}  

    #Verify candidate
    for c in candidates_set:
        list_const, selected_path, variablevalues = verify_candidate(solution_, c, source, v_type, row)  
        
        if len(list_const) == 0 and selected_path != None:   
            #Feasible candidate
            # cost function including node popularity
            cost_c = sum([(a*b)/c for a,b,c in zip(WeightFactors, variablevalues, NormalizationValues)]) + variablevalues[1]*(1-(len(candidates_sfcr[c])/sfcr_u)) 
            #original
            # cost_c = sum([(a*b)/c for a,b,c in zip(WeightFactors, variablevalues, NormalizationValues)])    
            
            feasible_cadidates[c] = [selected_path, variablevalues, cost_c]
        else:                                           
            unfeasible_candidates[c] = list_const    # save Unfeasible candidate 
        
    return feasible_cadidates, unfeasible_candidates

In [137]:
def mapping_procedure(solution_copy, row, available_c, len_sessions, flag_look_ahead, available_nc):
    """ 
    Thbis function is in charge of mapping the vnfs in the sfcr, ite will return a flag indicating the 
    failure or success of the mapping procedure and a dict containing the data  associated  to the output status"""
        
    flag_mapping_error = False 
    flag_resources_released = False
    VNFLoc = {}         #dict vnf_id as key and c, lprop as value
    
    for f_id, f_type  in zip(row['VNFs'], row['Type']):             # f , f_typev= vnf_id, vnf_type
        if f_id not in VNFLoc.keys():  
            # If the VNF has not been mapped yet, we need to analyze its placement solut.
            best_cost = 1000
            destination_vnf_f = []

            # steps 5-8:
            for branch, Lprop in zip(row['Presence'], row['L_prop']):
                # Steps 9-11: Select the candidates that are common to all the source according to Lprop.
                # In our specific case, the source is the same for all vnf. thus, we just need to update the candidates.
                if f_id == 1:
                    source_vnf_f = row['SourceBS']          # The BS is the initial source for all the branches
                    Lprop_vnf_f = [Lprop]                   # This is also updated with the placement of each VNf in a branch
                    candidates_f = row['Candidates']        # For the first vnf all candidates satisfy latency req
                else:
                    source_vnf_f = VNFLoc[1][0]    # esto es porq en el caso de anaísis es la misma source, first f
                    Lprop_vnf_f = VNFLoc[1][1]      # esto es porq en el caso de anaísis es la misma source, first f
                    # for previous_f_index in range(0,f_id-1):  # In case of having different sourcess
                    #     if branch[previous_f_index] == 1:
                    #         source = row['VNFs'][previous_f_index]
                    # source_vnf_f,append(VNFLoc[source][0])                                       
                    # Lprop_vnf_f.append(VNFLoc[source][1])
                    candidates_f = determine_candidates(source_vnf_f, Lprop_vnf_f, available_c)

                # determining if f is the last vnf in the chain: In our specific case, it is easier to see the type of the vnf.
                #  I it's the last, we do not need to determine the destination_vnfs
                if f_type != 1 and flag_look_ahead:               
                    for next_f_index in range(f_id,len(row['VNFs'])):  # f_idstarts in 1, thus if we used to iterate,       
                                                                    # we will be looking at the next vnfs.
                        if branch[next_f_index] == 1:     
                            destination_vnf_f.append(row['VNFs'][next_f_index])
                            break  
                            
            # steps 12-14: Evaluate candidates
            feasible_cadidates, unfeasible_candidates = evaluate_candidates(solution_copy,candidates_f,source_vnf_f,
                                                                            f_type, row, Candidates_sfcr,
                                                                            len_sessions) 
            # steps 17: sort feasible candidates by their cost in ascending order
            sorted_candidates_f = sorted(feasible_cadidates.keys(), key=lambda k: feasible_cadidates[k][2])
            
            for c in sorted_candidates_f:
                selected_path, variablevalues, cost = feasible_cadidates[c]
                if cost < best_cost:
                    # step 20 & 27: simulating deployment of vnf f
                    VNFLoc_aux = copy.deepcopy(VNFLoc)                                        
                    solution_copy2 = copy.deepcopy(solution_copy)
                    solution_copy2 = simulating_vnfdeployment(solution_copy2, c, f_id, f_type, selected_path, 
                                                              variablevalues, row)
                #steps 15-: Looking-ahead
                    if flag_look_ahead and f_type !=1:                    
                        # Step 21: Sort next VNFs  by criticism (Not implemented)
                        # Steps 22-: the following  first three lines should be for each next vnf but given that 
                        # they have similar features, I will do it just one time.
                        Lprop_next_f = [Lprop_vnf_f[0]-Paths[selected_path]] 
                        candidates_next_f = determine_candidates(c, Lprop_next_f, available_c)
                        VNFLoc_aux[f_id] = (c, Lprop_next_f) 
            
                        for next_f_id in destination_vnf_f:
                            # steps 24-25: Evaluate candidates
                            feasible_cadidates_next_f, unfeasible_candidates = evaluate_candidates(solution_copy2,
                                                                                            candidates_next_f, c, 
                                                                                            row['Type'][next_f_id-1],
                                                                                            row, Candidates_sfcr, 
                                                                                            len_sessions)             
                            if len(feasible_cadidates_next_f)!= 0:
                                # steps 26: selecting the best candidates for the next vnf                            
                                best_c_next_f = sorted(feasible_cadidates_next_f.keys(), key=lambda k: 
                                                             feasible_cadidates_next_f[k][2])[0]
                                
                                # Step 27: simulating deployment of next vnf
                                solution_copy2 = simulating_vnfdeployment(solution_copy2, best_c_next_f, next_f_id, 
                                                                          row['Type'][next_f_id-1],
                                                                          feasible_cadidates_next_f[best_c_next_f][0], 
                                                                          feasible_cadidates_next_f[best_c_next_f][1],
                                                                          row)
                                # Step 28: Update Cost
                                cost += feasible_cadidates_next_f[best_c_next_f][2]
                                VNFLoc_aux[next_f_id] = (best_c_next_f, Lprop_next_f[0]-
                                                         Paths[feasible_cadidates_next_f[best_c_next_f][0]])                                
                            else: 
                                cost += 1000    
                                break

                        if cost < best_cost:  # better candidate
                            best_cost = cost
                            best_solution = copy.deepcopy(solution_copy2)
                            VNFLoc_best = copy.deepcopy(VNFLoc_aux)      

                    else:
                        best_cost = feasible_cadidates[c][2] 
                        best_solution = copy.deepcopy(solution_copy2)
                        VNFLoc_best = copy.deepcopy(VNFLoc_aux) 
                        VNFLoc_best[f_id] = (c, [Lprop_vnf_f[0]-Paths[feasible_cadidates[c][0]]]) 
                        break     

            # steps 33-34:        
            if best_cost < 1000:
                solution_copy = copy.deepcopy(best_solution)
                VNFLoc = copy.deepcopy(VNFLoc_best)

            else:
                # PDT: Call reassign procedure()
                if not flag_look_ahead:
                    vnf_type = f_type   #pdt
                
                flag_mapping_error = True
                print('unfeasible_candidates', row['SessionId'], unfeasible_candidates)
                print('feasible_cadidates', row['SessionId'], feasible_cadidates)
                flag_resources_released, solution_copy = reassign_procedure(source_vnf_f, Lprop_vnf_f, vnf_type, 
                                                                            flag_look_ahead, solution_copy, row,
                                                                            available_nc, Candidates_sfcr,
                                                                            len_sessions)
                break
                        
    return flag_mapping_error, flag_resources_released, solution_copy

In [138]:
def reassign_procedure(source, lprop, cf_type, flag_look_ahead, solution_copy, row, available_nc, Candidates_sfcr,
                      len_sessions):   
    """
    This fucntion tries to map the SFCR that failed due to the absence of feasible candidates. This fucntionmay be uncompleted since
    we do not used it during our experiments.
    """

    # Step1- Evaluate the candidates by including the ones that were discarded  due to capacity limitation
    flag_resources_released = False
    candidates_r = determine_candidates(source, lprop, available_nc)
  
    feasible_cadidates_r, unfeasible_candidates_r = evaluate_candidates(solution_copy, candidates_r, source, cf_type, 
                                                                    row, Candidates_sfcr, len_sessions)

    # step2- select only the ones that are unfeasible due to capacity limitations only (C1 and7or C2). Selects
    # the ones that have C2
    selected_c = {}
    for c,list_const in unfeasible_candidates_r.items():
        print(c, list_const)
        if list_const == ['C1', 'C2'] or list_const == ['C2']:
            selected_c.append(c)

    # step3: 
    if len(selected_c)!=0:
        # step 4- define a set of untouchable candidates, the ones that are in the set of feasible candidates (para el approach
        # of looking ahead. Para no look ahead, no importa porque el estado des estos ya puede incluir otras vnfs de la 
        # cadena, es decir, no se afectaría.)
        untouchable_c = list(feasible_cadidates_r.keys())
        touchable_c = [c for c in available_nc if c not in untouchable_c]
        
        # step5: Sort selected candidates by clossness to the source vnf.    
        selected_c_distance_source = {}
        for c in selected_c:
            if (source,c,1) in Paths.keys():
                selected_c_distance_source[c]=Paths[(source,c,1)] 
        sort_selected_c_distance_source = sorted(selected_c_distance_source.items(), key=lambda x: x[1])
        
        # step6- Release resources in this candidate 
        print('Uncomplete function')
        raise

    return flag_resources_released, solution_copy

## Main Procedure

In [139]:
#Initializing variables
solution = {}
solution['ServersCap'] = copy.deepcopy(capacity_nc)
solution['LinksCap'] = copy.deepcopy(capacity_links)
solution['OpenServers'] = []
solution['VNFInstances'] = {}  # the key is the type of VNF and the value is a list of instances id
solution['VNFCapacity'] = {}   # dict of dict, main dict key=type, valueis a dict key=instance_id & value=available cap
solution['NodeVNFInstances'] = {} # dict of dict, main dict key=node id, valueis a dict key=vnf_type & value=instance id
solution['VNFSFCAssigned'] = {} # dict of dict, main dict key=type, valueis a dict key=instance_id & value=sfc id
solution['SFCMapping'] = {} # dict of dict, main dict key=pdu id, value= dict key=vnf_id(f)& value=type,instance_id,nodeId
SFCUnmapped = []
Count_reassig = 0

SessionInfo = critic_classifier(SessionInfo, Available_nc)
SortedSessionInfo = copy.deepcopy(SessionInfo)
Available_c = copy.deepcopy(Available_nc)

#Sorting SFCR 
SortedSessionInfo = SortedSessionInfo.sort_values(by=['Critic', 'Latency', 'NumVNFs', 'NumCand', 'SourceBS'],
                                            ascending=[False, True, False, True, True])
                                            
flag_look_ahead = True
# flag_look_ahead = import_data(dir_data, "flag_look_ahead") 
flag_reasing = False   #no implementado

start_time = time.process_time()

#Determining foreach candidates the sfcr that it can serve
Candidates_sfcr = {}
for c in Available_c:
    Candidates_sfcr[c] = []
for index, row in SessionInfo.iterrows():
    for c in row['Candidates']:
        Candidates_sfcr[c].append(row['SessionId'])

while len(SortedSessionInfo) != 0:
    solution_copy = copy.deepcopy(solution)
    row = SortedSessionInfo.iloc[0]   #selecting the first SFCR in the sorted  set
    len_sessions = len(SortedSessionInfo)
    
    # Procedure: mapping
    flag_mapping_error, flag_resources_released, solution_copy = mapping_procedure(solution_copy, row, Available_c, 
                                                                                   len_sessions, flag_look_ahead, 
                                                                                   Available_nc)
    # Procedure: reassign
    if flag_mapping_error and flag_resources_released:
        raise
        flag_reasing = False
        Count_reassig += 1
    
    # steps 40- : Update resources and (un)mapping set            
    if not flag_mapping_error or flag_reasing:  
        solution = copy.deepcopy(solution_copy)
        #checking if we need o delete some candidate
        flag_sort_sfcr = False

        for c, instances in solution['NodeVNFInstances'].items():
            if solution['ServersCap'][c] == 0 and c in Available_c:   # Analize available capacity of VNFs in c
                flag_update_c = True
                for typ,id_f in instances.items():
                    for f in id_f:
                        if solution['VNFCapacity'][typ][f] > 0:
                            flag_update_c = False
                            break
                    if not flag_update_c: break
                if flag_update_c:
                    flag_sort_sfcr = True
                    Available_c = [i for i in Available_c if i!=c]

        if flag_sort_sfcr:
            SortedSessionInfo = critic_classifier(SortedSessionInfo, Available_c)
            #Sorting SFCR 
            SortedSessionInfo = SortedSessionInfo.sort_values(by=['Critic', 'Latency', 'NumVNFs', 'NumCand', 'SourceBS'],
                                                                ascending=[False, True, False, True, True])        
    else:
        SFCUnmapped.append(row['SessionId'])              
        
    # updating the set of SFCR and Candidates_sfcr
    SortedSessionInfo = SortedSessionInfo.drop(SortedSessionInfo.index[0]) 
    for k,v in Candidates_sfcr.items(): 
        if row['SessionId'] in v: Candidates_sfcr[k].remove(row['SessionId'])
    
T_exec = time.process_time()-start_time

## Processing the output results

In [ ]:
def get_cost(solution,WeightFactors, NormalizationValues):
    """Calculates cost of the solution according to their weight in the objective function."""
    
    num_open_servers = len(solution['OpenServers'])
    num_upf_instances = sum([len(v) for v in solution['VNFInstances'].values()])
    
    latency_prop = 0
    branches = 0
    for k,v in solution['SFCMapping'].items():
        for branch in SessionInfo[SessionInfo['SessionId']==k]['Presence'].values[0]:
            latency_branch = sum([Paths[a]*b for a,b in zip(v['Paths'],branch)])    # one way 
            latency_prop += latency_branch
            branches += 1

    variable_values = [num_open_servers, num_upf_instances, latency_prop/branches]
    cost = sum([a*b/c for a,b,c in zip(WeightFactors,variable_values,NormalizationValues)])
    # print(' Cost_evaluation :' , cost)
    
    return cost

In [140]:
Total_upf = 0
Num_upf_type = {}

for k,v in solution['VNFInstances'].items():
    Total_upf += len(v)
    Num_upf_type[k] = len(v)
    
# Number of open servers
Num_open_c = len(solution['OpenServers'])
Open_servers_list = solution['OpenServers']

#Number of Unassigned sessions 
Num_unassigned_s = len(SFCUnmapped)
Unassigned_s_list = SFCUnmapped


# Laltency
Latency_List = []

for index, row in SessionInfo.iterrows():
    if row['SessionId'] in solution['SFCMapping'].keys():
        if row['Branches']==2:
            Lprop_m =[Paths[solution['SFCMapping'][row['SessionId']]['Paths'][0]]+Paths[solution['SFCMapping'][row['SessionId']]['Paths'][1]]
                      ,Paths[solution['SFCMapping'][row['SessionId']]['Paths'][0]]+Paths[solution['SFCMapping'][row['SessionId']]['Paths'][2]]]
            for branch, Lprop_b, Lprop in zip(row['Presence'], row['L_prop'], Lprop_m ):
        #         print(Lprop, branch)
                Lproc = row['Latency']-2*Lprop_b
                Latency_List.append(Lproc+2*Lprop)
        else:
            Lproc = row['Latency']-2*row['L_prop'][0] 
            Lprop = 0
            for path in solution['SFCMapping'][row['SessionId']]['Paths']:
                Lprop += Paths[path]

            Latency_List.append(Lproc+2*Lprop)

In [143]:
cost = get_cost(solution,WeightFactors, NormalizationValues)

 Cost_evaluation : 0.34595445997975854


In [113]:
print('   Execution time', T_exec)
print('   Placement Cost', cost)
print('   Number of UPF installed: ', Total_upf )
print('   Number of UPFs per type', Num_upf_type )
print('   Open servers list', Open_servers_list )
print('   Number of Open Servers', Num_open_c )
print('   Num_unassigned sessions', Num_unassigned_s)
print('   Number of Reassignations', Count_reassig )

   Execution time 1.34375
   Placement Cost 0.3219933528816182
   Number of UPF installed:  18
   Number of UPFs per type {3: 4, 1: 11, 2: 3}
   Open servers list [5, 4, 1, 8, 3, 9]
   Number of Open Servers 6
   Num_unassigned sessions 0
   Number of Reassignations 0
